In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_iris
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split

In [19]:
iris = load_iris()
iris
df = pd.DataFrame(iris.data,columns=iris.feature_names)
df['target'] = iris.target
df.sample(10)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
124,6.7,3.3,5.7,2.1,2
91,6.1,3.0,4.6,1.4,1
29,4.7,3.2,1.6,0.2,0
40,5.0,3.5,1.3,0.3,0
128,6.4,2.8,5.6,2.1,2
36,5.5,3.5,1.3,0.2,0
49,5.0,3.3,1.4,0.2,0
0,5.1,3.5,1.4,0.2,0
21,5.1,3.7,1.5,0.4,0
125,7.2,3.2,6.0,1.8,2


In [20]:
train_df,test_df = train_test_split(df,test_size=0.2,random_state=42,stratify=df['target'])

In [21]:
X_train = train_df.drop(columns=['target'])
X_test = test_df.drop(columns=['target'])

y_train = train_df['target']
y_test = test_df['target']

In [15]:
print(X_train.shape)
print(y_train.shape)

(120, 4)
(120,)


In [23]:
scalar = StandardScaler()
X_train = scalar.fit_transform(X_train)
X_test = scalar.transform(X_test)

In [24]:
X_train_tensor = torch.tensor(X_train,dtype=torch.float32)
X_test_tensor = torch.tensor(X_test,dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values,dtype=torch.long)
y_test_tensor = torch.tensor(y_test.values,dtype=torch.long)

In [25]:
class IrisClassifier(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size):
        super().__init__()
        self.network = nn.Sequential(
            nn.Linear(input_size, hidden_size1),
            nn.ReLU(),
            nn.Linear(hidden_size1, hidden_size2),
            nn.ReLU(),
            nn.Linear(hidden_size2, output_size)
        )

    def forward(self, x):
        return self.network(x)


In [26]:
X_train.shape

(120, 4)

In [27]:
input_size = X_train.shape[1]
hidden_size1 = 32
hidden_size2 = 16
output_size = 3
model = IrisClassifier(input_size,hidden_size1,hidden_size2,output_size)

In [28]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=0.01)

In [30]:
epochs = 500
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    predictions = model(X_train_tensor)
    loss = criterion(predictions,y_train_tensor)
    loss.backward()
    optimizer.step()

    if (epoch+1)%50 == 0:
        print(f"Epoch: {epoch + 1}/{epochs} Loss: {loss.item():.4f}")


Epoch: 50/500 Loss: 0.0525
Epoch: 100/500 Loss: 0.0259
Epoch: 150/500 Loss: 0.0103
Epoch: 200/500 Loss: 0.0032
Epoch: 250/500 Loss: 0.0013
Epoch: 300/500 Loss: 0.0007
Epoch: 350/500 Loss: 0.0004
Epoch: 400/500 Loss: 0.0003
Epoch: 450/500 Loss: 0.0002
Epoch: 500/500 Loss: 0.0001


In [35]:
model.eval()
with torch.no_grad():
    y_pred = model(X_test_tensor)
    predicted = torch.argmax(y_pred, dim=1)
    
    correct = (predicted == y_test_tensor).sum().item()
    total = y_test_tensor.size(0)
    accuracy = correct / total

    print(f"Accuracy: {accuracy * 100:.2f}%")


Accuracy: 96.67%


In [43]:
def predict_iris(s_l,s_w,p_l,p_w):
    input_data = np.array([[s_l,s_w,p_l,p_w]])
    input_data = scalar.transform(input_data)
    input_tensor = torch.tensor(input_data,dtype=torch.float32)
    
    model.eval()
    with torch.no_grad():
        prediction = model(input_tensor)
        input_data_pred = torch.argmax(prediction,dim=1).item()
    return iris.target_names[input_data_pred]

In [44]:
predict_iris(5.1,3.5,1.4,0.2)

c:\Users\ANAND REDDY\anaconda3\envs\resumeEnv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


'setosa'